Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.552203
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.258575
Minibatch accuracy: 56.2%
Validation accuracy: 62.5%
Minibatch loss at step 100: 1.088562
Minibatch accuracy: 81.2%
Validation accuracy: 75.0%
Minibatch loss at step 150: 0.239195
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 200: 0.398868
Minibatch accuracy: 93.8%
Validation accuracy: 79.1%
Minibatch loss at step 250: 0.533116
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 300: 0.298291
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 350: 0.582839
Minibatch accuracy: 81.2%
Validation accuracy: 77.1%
Minibatch loss at step 400: 0.651347
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.129884
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.555028
Minibatch accuracy: 93.8%
Validation accuracy: 81.8%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [40]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
#   layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer2_biases = tf.Variable(tf.zeros([depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer1_biases)

    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.436567
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.613257
Minibatch accuracy: 43.8%
Validation accuracy: 48.7%
Minibatch loss at step 100: 1.075494
Minibatch accuracy: 75.0%
Validation accuracy: 71.8%
Minibatch loss at step 150: 0.363268
Minibatch accuracy: 93.8%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.378605
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
Minibatch loss at step 250: 0.687587
Minibatch accuracy: 75.0%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.355322
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.453309
Minibatch accuracy: 81.2%
Validation accuracy: 75.5%
Minibatch loss at step 400: 0.643692
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.079533
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.669765
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

INPUT -> [CONV -> RELU -> POOL]*2 -> FC -> RELU -> FC

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    conv_weights_1 = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    conv_biases_1 = tf.Variable(tf.zeros([depth]))
    
    conv_weights_2 = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    conv_biases_2 = tf.Variable(tf.zeros([depth]))
    
    fc_weights_1 = tf.Variable(tf.truncated_normal(
        [image_size * image_size * depth // 16, num_hidden], stddev=0.1))
    fc_biases_1 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    fc_weights_2 = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    fc_biases_2 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    def conv_relu_pool(in_data, weights, biases):    
        conv = tf.nn.conv2d(in_data, weights, [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + biases)
        pool = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        return pool
    
    def fc(in_data, weights, biases):
        shape = in_data.get_shape().as_list()
        if len(shape) == 4:
            reshape = tf.reshape(in_data, [shape[0], shape[1] * shape[2] * shape[3]])
        else:
            reshape = in_data

        return tf.add(tf.matmul(reshape, weights), biases)
    
    def fc_relu(in_data, weights, biases):
        return tf.nn.relu(fc(in_data, weights, biases))

    def model(in_data):
        hidden = conv_relu_pool(in_data, conv_weights_1, conv_biases_1)
        hidden = conv_relu_pool(hidden, conv_weights_2, conv_biases_2)
        hidden = fc_relu(hidden, fc_weights_1, fc_biases_1)
        return fc(hidden, fc_weights_2, fc_biases_2)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [55]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.261415
Minibatch accuracy: 12.5%
Validation accuracy: 11.6%
Minibatch loss at step 50: 0.679256
Minibatch accuracy: 81.2%
Validation accuracy: 71.8%
Minibatch loss at step 100: 1.510088
Minibatch accuracy: 50.0%
Validation accuracy: 71.3%
Minibatch loss at step 150: 0.244855
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 200: 0.241482
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 250: 0.373137
Minibatch accuracy: 93.8%
Validation accuracy: 82.2%
Minibatch loss at step 300: 0.225635
Minibatch accuracy: 93.8%
Validation accuracy: 81.2%
Minibatch loss at step 350: 0.420633
Minibatch accuracy: 93.8%
Validation accuracy: 81.7%
Minibatch loss at step 400: 0.479381
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 450: 0.185641
Minibatch accuracy: 100.0%
Validation accuracy: 83.5%
Minibatch loss at step 500: 0.794526
Minibatch accuracy: 68.8%
Validation accuracy: 82.1%

INPUT -> [CONV -> RELU -> CONV -> RELU -> POOL] * 3 -> [FC -> RELU] * 2 -> FC

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 128

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    
    conv_weights_1_1 = weight_variable([patch_size, patch_size, num_channels, depth])
    conv_biases_1_1 = bias_variable([depth])
    
    conv_weights_1_2 = weight_variable([patch_size, patch_size, depth, depth])
    conv_biases_1_2 = bias_variable([depth])
    
    conv_weights_2_1 = weight_variable([patch_size, patch_size, depth, depth])
    conv_biases_2_1 = bias_variable([depth])
    
    conv_weights_2_2 = weight_variable([patch_size, patch_size, depth, depth])
    conv_biases_2_2 = tf.Variable(tf.zeros([depth]))
    
    conv_weights_3_1 = weight_variable([patch_size, patch_size, depth, depth])
    conv_biases_3_1 = tf.Variable(tf.zeros([depth]))
    
    conv_weights_3_2 = weight_variable([patch_size, patch_size, depth, depth])
    conv_biases_3_2 = bias_variable([depth])
    
    fc_weights_1 = weight_variable([image_size * image_size * depth // 49, num_hidden])
    fc_biases_1 = bias_variable([num_hidden])
    
    fc_weights_2 = weight_variable([num_hidden, num_hidden])
    fc_biases_2 = bias_variable([num_hidden])
    
    fc_weights_3 = weight_variable([num_hidden, num_labels])
    fc_biases_3 = bias_variable([num_labels])
    
    
    def conv_relu(in_data, weights, biases):
        conv = tf.nn.conv2d(in_data, weights, [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + biases)
        return relu
        
    def conv_relu_pool(in_data, weights, biases, pool_stride=2):    
        relu = conv_relu(in_data, weights, biases)
        pool = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, pool_stride, pool_stride, 1], padding='SAME')
        return pool
    
    def fc(in_data, weights, biases):
        shape = in_data.get_shape().as_list()
        if len(shape) == 4:
            reshape = tf.reshape(in_data, [shape[0], shape[1] * shape[2] * shape[3]])
        else:
            reshape = in_data

        return tf.add(tf.matmul(reshape, weights), biases)
    
    def fc_relu(in_data, weights, biases, dropout=True):
        relu = tf.nn.relu(fc(in_data, weights, biases))

        if dropout:
            return tf.nn.dropout(relu, 0.5)
        else:
            return relu

    def model(in_data):
        hidden = conv_relu(in_data, conv_weights_1_1, conv_biases_1_1)
        hidden = conv_relu_pool(hidden, conv_weights_1_2, conv_biases_1_2)
        
        hidden = conv_relu(hidden, conv_weights_2_1, conv_biases_2_1)
        hidden = conv_relu_pool(hidden, conv_weights_2_2, conv_biases_2_2)
        
        hidden = conv_relu(hidden, conv_weights_3_1, conv_biases_3_1)
        hidden = conv_relu_pool(hidden, conv_weights_3_2, conv_biases_3_2)
        
        hidden = fc_relu(hidden, fc_weights_1, fc_biases_1)
        
        hidden = fc_relu(hidden, fc_weights_2, fc_biases_2)
        
        return fc(hidden, fc_weights_3, fc_biases_3)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.05, global_step, 100, 0.95, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.985924
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 100: 1.883303
Minibatch accuracy: 43.8%
Validation accuracy: 25.9%
Minibatch loss at step 200: 1.764879
Minibatch accuracy: 43.8%
Validation accuracy: 44.7%
Minibatch loss at step 300: 1.243842
Minibatch accuracy: 56.2%
Validation accuracy: 57.6%
Minibatch loss at step 400: 0.538915
Minibatch accuracy: 75.0%
Validation accuracy: 66.6%
Minibatch loss at step 500: 0.833622
Minibatch accuracy: 68.8%
Validation accuracy: 71.3%
Minibatch loss at step 600: 1.138888
Minibatch accuracy: 75.0%
Validation accuracy: 74.5%
Minibatch loss at step 700: 1.113178
Minibatch accuracy: 68.8%
Validation accuracy: 76.9%
Minibatch loss at step 800: 0.580159
Minibatch accuracy: 87.5%
Validation accuracy: 78.3%
Minibatch loss at step 900: 0.613143
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 0.335653
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

image_size = 28
num_labels = 10
num_channels = 1

batch_size = 50
patch_size = 5

graph = tf.Graph()

with graph.as_default():
    # Input data.
    X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_train_dataset = tf.reshape(X, [-1, image_size, image_size, 1])
    tf_train_labels = y
    
    tf_test_dataset = tf.constant(mnist.test.images.reshape((-1, image_size, image_size, 1)).astype(np.float32))

    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    
    conv_weights_1 = weight_variable([patch_size, patch_size, num_channels, 32])
    conv_biases_1 = bias_variable([32])
    
    conv_weights_2 = weight_variable([patch_size, patch_size, 32, 64])
    conv_biases_2 = bias_variable([64])
    
    fc_weights_1 = weight_variable([7 * 7 * 64, 1024])
    fc_biases_1 = bias_variable([1024])
    
    fc_weights_2 = weight_variable([1024, num_labels])
    fc_biases_2 = bias_variable([num_labels])
    
    
    def conv_relu(in_data, weights, biases):
        conv = tf.nn.conv2d(in_data, weights, [1, 1, 1, 1], padding='SAME')
        relu = tf.nn.relu(conv + biases)
        return relu
        
    def conv_relu_pool(in_data, weights, biases, pool_stride=2):    
        relu = conv_relu(in_data, weights, biases)
        pool = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, pool_stride, pool_stride, 1], padding='SAME')
        return pool
    
    def fc(in_data, weights, biases):
        shape = in_data.get_shape().as_list()
        if len(shape) == 4:
            reshape = tf.reshape(in_data, [shape[0], shape[1] * shape[2] * shape[3]])
        else:
            reshape = in_data

        return tf.add(tf.matmul(reshape, weights), biases)
    
    def fc_relu(in_data, weights, biases, dropout=True):
        relu = tf.nn.relu(fc(in_data, weights, biases))

        if dropout:
            return tf.nn.dropout(relu, 0.5)
        else:
            return relu

    def model(in_data, dropout=True):
        hidden = conv_relu_pool(in_data, conv_weights_1, conv_biases_1)
        hidden = conv_relu_pool(hidden, conv_weights_2, conv_biases_2)

        hidden = fc_relu(hidden, fc_weights_1, fc_biases_1, dropout=dropout) 
        
        return fc(hidden, fc_weights_2, fc_biases_2)
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.001, global_step, 50, 0.95, staircase=True)

    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout=False))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [9]:
num_steps = 20000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        batch = mnist.train.next_batch(batch_size)
        feed_dict = {X : batch[0], y : batch[1]}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch[1]))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), mnist.test.labels))

Initialized
Minibatch loss at step 0: 14.665867
Minibatch accuracy: 6.0%
Minibatch loss at step 100: 0.296139
Minibatch accuracy: 90.0%
Minibatch loss at step 200: 0.133688
Minibatch accuracy: 96.0%
Minibatch loss at step 300: 0.360232
Minibatch accuracy: 94.0%
Minibatch loss at step 400: 0.072587
Minibatch accuracy: 96.0%
Minibatch loss at step 500: 0.072940
Minibatch accuracy: 96.0%
Minibatch loss at step 600: 0.137592
Minibatch accuracy: 98.0%
Minibatch loss at step 700: 0.097062
Minibatch accuracy: 96.0%
Minibatch loss at step 800: 0.037244
Minibatch accuracy: 100.0%
Minibatch loss at step 900: 0.345505
Minibatch accuracy: 90.0%
Minibatch loss at step 1000: 0.037295
Minibatch accuracy: 98.0%
Minibatch loss at step 1100: 0.029315
Minibatch accuracy: 100.0%
Minibatch loss at step 1200: 0.149196
Minibatch accuracy: 92.0%
Minibatch loss at step 1300: 0.128545
Minibatch accuracy: 94.0%
Minibatch loss at step 1400: 0.082994
Minibatch accuracy: 96.0%
Minibatch loss at step 1500: 0.014903
